In [ ]:
import ROOT

In [ ]:
configs = [
    "BDT_Grad_2",
    "BDT_Grad_3",
    "BDT_Grad_4",
]
bdt_files = {
    c: ROOT.TFile.Open(f"{c}/tmva_output.root") for c in configs
}

In [ ]:
bdt_outputs = {
    c: f.Get(f"{c}/Method_BDT/BDT") for c, f in bdt_files.items()
}

In [ ]:
ROOT.gROOT.ForceStyle()
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPalette(ROOT.kRainBow)

In [ ]:
hists = {}
roc = {}
for c, f in bdt_outputs.items():
    hists[f"{c}_mva_s"] = f.Get("MVA_BDT_S")
    hists[f"{c}_mva_b"] = f.Get("MVA_BDT_B")
    hists[f"{c}_mva_s_train"] = f.Get("MVA_BDT_Train_S")
    hists[f"{c}_mva_b_train"] = f.Get("MVA_BDT_Train_B")

    roc[f"{c}_effBvsS"] = f.Get("MVA_BDT_effBvsS")
    roc[f"{c}_effBvsS_train"] = f.Get("MVA_BDT_trainingEffBvsS")

In [ ]:
draw_mva = """
canvas = ROOT.TCanvas()
legend = ROOT.TLegend(0.6, 0.7, 0.88, 0.87, f"{c}")
legend.SetNColumns(2)
hs = hists[f"{c}_mva_s"]
hst = hists[f"{c}_mva_s_train"]
hb = hists[f"{c}_mva_b"]
hbt = hists[f"{c}_mva_b_train"]
hs.SetLineColor(ROOT.kBlue)
hst.SetMarkerColor(ROOT.kBlue)
hst.SetMarkerStyle(7)
hb.SetLineColor(ROOT.kRed)
hbt.SetLineColor(ROOT.kRed)
legend.AddEntry(hs, "Signal", "l")
legend.AddEntry(hst, "Signal (Train)", "ep")
legend.AddEntry(hb, "Bkg", "l")
legend.AddEntry(hbt, "Bkg (Train)", "ep")
frame = max(hs, hst, hb, hbt, key=lambda x: x.GetMaximum()).Clone("frame")
frame.SetMaximum(1.4 * frame.GetMaximum())
frame.SetTitle(f"{c};BDT Score;")
frame.Draw("axis")
hs.Draw("hist same")
hst.Draw("x0 p same")
hb.Draw("hist same")
hbt.Draw("x0 p same")
legend.Draw()
canvas.Draw()
"""

In [ ]:
c = "BDT_Grad_2"
exec(draw_mva)

In [ ]:
c = "BDT_Grad_3"
exec(draw_mva)

In [ ]:
c = "BDT_Grad_4"
exec(draw_mva)

In [ ]:
draw_roc = """
canvas = ROOT.TCanvas()
frame = canvas.DrawFrame(0.0, 0.0, 1.0, 1.0, ";sig eff;bkg eff")
frame.Draw()
legend = ROOT.TLegend(0.12, 0.6, 0.3, 0.88)
for c in configs:
    r = roc[f"{c}_effBvsS"]
    r.Draw("same C PLC")
    auc = 1 - (r.Integral()/100)
    legend.AddEntry(r, f"#splitline{{{c}}}{{({auc:.3f})}}", "l")
legend.Draw()
canvas.SetLogy(0)
canvas.Draw()

"""
exec(draw_roc)